<a href="https://colab.research.google.com/github/LuisPeMoraRod/AI-Laboratories/blob/main/Lab7_ErrorRates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ejercicio 1.
Genere un set de datos con comportamiento lineal (puede utilizar una librería para generar este set) de 100 mil datos, calcule la regresión lineal. Puede reutilizar los datos del lab de Regresión Lineal (5 pts)

In [1]:
# import libraries
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt #for plotting
import torch
import numpy as np

In [2]:
N_SAMPLES = 100000 #number of samples
N_FEATURES = 1 #number of features
N_INFORMATIVE = 1 #number of useful features
NOISE = 10 #bias and standard deviation of the guassian noise
x, y, coef = make_regression(n_samples=N_SAMPLES,
                            n_features=N_FEATURES,
                            n_informative=N_INFORMATIVE, 
                            noise=NOISE,
                            coef=True, # return the coefficients of the underlying linear model
                            random_state=0) #set for same data points for each run

# Scale feature x (students' height) to range [1.20, 1.40]
x = np.interp(x, (x.min(), x.max()), (1.20, 1.40))
x_t = np.transpose(x)[0]
x_data = torch.tensor(x_t, dtype=torch.float64)

# Scale target y (students' weight) to range [25,40] 
y = np.interp(y, (y.min(), y.max()), (25, 40))
y_data = torch.tensor(y, dtype=torch.float64)

print(x_data)
print(y_data)

tensor([1.3287, 1.2918, 1.3016,  ..., 1.3029, 1.3380, 1.3173],
       dtype=torch.float64)
tensor([34.1849, 31.4623, 32.6545,  ..., 32.5608, 34.9858, 33.6121],
       dtype=torch.float64)


### Ejercicio 2
Calcule diferentes índices de error sobre la regresión. Los algoritmos deben ser programados por usted. No se permite que utilice métodos ya programados de librerías. 

In [3]:
def linearRegression (array_X,array_Y):
  #x and y average
  x_avg = torch.mean(array_X)
  y_avg = torch.mean(array_Y)

  #data needed to calculate b and a
  xi_xm = torch.add(array_X, -(x_avg))
  yi_ym = torch.add(array_Y, -(y_avg))

  num = torch.mul(xi_xm,yi_ym)
  den = torch.pow(xi_xm,2)

  num_sum = torch.sum(num)
  den_sum = torch.sum(den)

  #Linear regression y = a + bx
  #b calculation
  b = num_sum / den_sum
  #a calculation
  a = y_avg - (b*x_avg)
  
  return (a,b)

a, b = linearRegression(x_data, y_data)
print(f'a = {a}')
print(f'b = {b}')

a = -65.28086377653793
b = 75.11420294360995


#### a) Calcule el Error Medio Absoluto (MAE). (20 pts)  

In [ ]:
# MAE method

#### b) Calcule el Error Medio Cuadrático (MSE). (20 pts) 

In [ ]:
# MSE method

#### c) Calcule el Root Mean Squared Error (RMSE). (20 pts) 

In [ ]:
# RMSE method